In [2]:
# import miscellaneous modules
import matplotlib.pyplot as plt
import os
import numpy as np
import time
import keras
import tensorflow as tf
import tensorflow_datasets as tfds
keras.__version__
tf.__version__

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2024-10-03 11:10:49.007435: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 11:10:49.018587: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 11:10:49.035152: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 11:10:49.041822: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 11:10:49.051075: I tensorflow/core/platform/cpu_feature_guar

1 Physical GPUs, 1 Logical GPU


I0000 00:00:1727928651.463103   37322 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727928651.506208   37322 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727928651.506247   37322 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727928651.509860   37322 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727928651.509906   37322 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [3]:
(train_ds, test_ds), info = tfds.load("fashion_MNIST", split=['train[:20%]', 'test[:20%]'],
                                      as_supervised=True, 
                                      with_info=True)

In [18]:
#info

In [4]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [6]:
len(train_ds)

12000

***ถ้าอยากจะ เปลี่ยน datatype ของ label จาก tf.int46 ให้เป็น tf.int8 ต้องทำอย่างไร*** 

จงปรับแก้ฟังก์ชัน nomalisation ที่ให้มานี้เปลี่ยน datatype ของ input_labels ด้วย สามารถใช้ tf.cast() เพื่อปรับ datatype ได้

***ฟังก์ชันอื่นที่ tf นั้นมีให้มีตัวไหนอีก??***

อยากให้ลองเรียก ฟังก์ชัน tf.image.random.... อื่นๆที่ tensorflow นั้นมีให้

In [8]:
def normalisation(input_imgs, input_labels):
    input_imgs = tf.cast(input_imgs, tf.float32) / 255.0
    return input_imgs, input_labels

def augmentation(input_imgs, input_labels):
    input_imgs = tf.image.random_flip_left_right(input_imgs)
    return input_imgs, input_labels

def prepare(input_imgs, input_labels):
    input_imgs, input_labels = normalisation(input_imgs, input_labels)
    input_imgs, input_labels = augmentation(input_imgs, input_labels)
    return input_imgs, input_labels

augmented_train_ds = train_ds.map(prepare).batch(32)

In [9]:
augmented_train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [10]:
model = tf.keras.Sequential([
  # Add the preprocessing layers you created earlier.
  tf.keras.Input(shape=(28,28,1)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10,activation="softmax")
])

In [11]:
model.compile(optimizer=keras.optimizers.Adam(), 
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy']
             )
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,578 (787.41 KB)

 Trainable params: 201,578 (787.41 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(augmented_train_ds, epochs=5)

Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9028 - loss: 0.2816
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9068 - loss: 0.2620
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9095 - loss: 0.2458
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9146 - loss: 0.2325
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9193 - loss: 0.2199
